In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
ruta = 'https://raw.githubusercontent.com/cesargar1507/DatasetsUB/main/worlddometer_covid_ue.csv?_sm_au_=iVVwgRnFF56ntMqT26q8vKHV6CTMc'
datos = pd.read_csv(ruta, sep = ';')
df = pd.DataFrame(datos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Num                     49 non-null     int64  
 1   Country                 49 non-null     object 
 2   Total Cases             49 non-null     object 
 3   New Cases               17 non-null     object 
 4   Total Deaths            46 non-null     object 
 5   New Deaths              7 non-null      float64
 6   Total Recovered         45 non-null     object 
 7   Active Cases            45 non-null     object 
 8   Serious Critical        37 non-null     object 
 9   Total Cases per 1M pop  48 non-null     object 
 10  Deaths per 1M pop       45 non-null     object 
 11  Total Tests             47 non-null     object 
 12  Tests per 1M pop        47 non-null     object 
 13  Population              48 non-null     object 
dtypes: float64(1), int64(1), object(12)
memory u

In [10]:
df.rename(columns={'Population ': 'Population'}, inplace = True)

In [11]:
if df['Total Deaths'].dtype == object:
  df['Total Deaths'] = pd.to_numeric(df['Total Deaths'].str.replace(',',''), errors= 'ignore')
else:
  print("La variable ya se ha convertido a numérica")

if df['Total Cases'].dtype == object:
  df['Total Cases'] = pd.to_numeric(df['Total Cases'].str.replace(',',''), errors= 'ignore')
else:
  print("La variable ya se ha convertido a numérica")

if df['Total Recovered'].dtype == object:
  df['Total Recovered'] = pd.to_numeric(df['Total Recovered'].str.replace(',',''), errors= 'ignore')
else:
  print("La variable ya se ha convertido a numérica")

if df['Total Tests'].dtype == object:
  df['Total Tests'] = pd.to_numeric(df['Total Tests'].str.replace(',',''), errors= 'ignore')
else:
  print("La variable ya se ha convertido a numérica")

if df['Population'].dtype == object:
  df['Population'] = pd.to_numeric(df['Population'].str.replace(',',''), errors= 'ignore')
else:
  print("La variable ya se ha convertido a numérica")


# **LITERAL 1**

In [28]:
paises_filtrados = ['Spain', 'Italy', 'UK', 'France', 'Germany', 'Ukraine']
df_filtrado = df[df['Country'].isin(paises_filtrados)].copy()
df_filtrado['prob_surv'] = 1-df_filtrado['Total Deaths']/df_filtrado['Total Cases']
df_filtrado[['Country','Total Cases','Total Deaths', 'Total Recovered','prob_surv']]

,Country,Total Cases,Total Deaths,Total Recovered,prob_surv
2,Spain,455621,29011.0,NaN,0.936326
3,UK,332752,41498.0,NaN,0.875289
4,France,272530,30602.0,86177.0,0.887711
5,Italy,266853,35473.0,208224.0,0.867069
6,Germany,242825,9363.0,217484.0,0.961441
7,Ukraine,119074,2527.0,56734.0,0.978778


# **LITERAL 2**

In [50]:
# Laplace: casos posibles entre totales - probabilidad de muerte o no muerte
df_filtrado['prob_mort'] =  df_filtrado['Total Deaths']/df_filtrado['Total Cases']
df_filtrado['prob_surv'] = 1- ( df_filtrado['prob_mort'])
# Laplace: casos posibles entre totales - probabilidad de cada pais
df_filtrado['Total_Country'] = df_filtrado['Total Cases'].sum()
df_filtrado['prob_pais'] = df_filtrado['Total Cases'] /df_filtrado['Total_Country']
df_bayes = df_filtrado[['Country','Total Cases','Total Deaths','prob_surv','prob_mort','prob_pais']].copy()
df_filtrado[['Country','Total Cases','Total Deaths','prob_surv','prob_mort','prob_pais']]

,Country,Total Cases,Total Deaths,prob_surv,prob_mort,prob_pais
2,Spain,455621,29011.0,0.936326,0.063674,0.269653
3,UK,332752,41498.0,0.875289,0.124711,0.196935
4,France,272530,30602.0,0.887711,0.112289,0.161293
5,Italy,266853,35473.0,0.867069,0.132931,0.157933
6,Germany,242825,9363.0,0.961441,0.038559,0.143713
7,Ukraine,119074,2527.0,0.978778,0.021222,0.070472


In [39]:
#Teorema de probabilidad total
prob_mort_total = (df_filtrado['prob_mort']*df_filtrado['prob_pais']).sum()
prob_surv = (1-prob_mort_total)*100
print ('Probabilidad de sobrevivir en cualquier país:', prob_surv.round(2) ,"%")

Probabilidad de sobrevivir en cualquier país: 91.21 %


In [55]:
#Teorema de Bayes total
df_bayes['prob_surv_pais'] = 1-(df_bayes['prob_mort']*df_bayes['prob_pais'])/prob_mort_total
df_bayes

,Country,Total Cases,Total Deaths,prob_surv,prob_mort,prob_pais,bayes,prob_surv_pais
2,Spain,455621,29011.0,0.936326,0.063674,0.269653,0.804606,0.804606
3,UK,332752,41498.0,0.875289,0.124711,0.196935,0.720503,0.720503
4,France,272530,30602.0,0.887711,0.112289,0.161293,0.793890,0.793890
5,Italy,266853,35473.0,0.867069,0.132931,0.157933,0.761083,0.761083
6,Germany,242825,9363.0,0.961441,0.038559,0.143713,0.936938,0.936938
7,Ukraine,119074,2527.0,0.978778,0.021222,0.070472,0.982980,0.982980


# **LITERAL 3**